#Upload dataset from Kaggle

In [ ]:
! pip install kaggle

In [ ]:
#For Colab
'''''
To obtain the Kaggle.json file:

1. Go to Kaggle, to your account, Scroll to API section and Click Expire API Token to remove previous tokens

2. Click on Create New API Token - It will download kaggle.json file on your machine.

3. Go to your Google Colab project file and run the following commands:

More info: https://www.kaggle.com/general/74235

'''''
from google.colab import files

files.upload()

In [ ]:
! cp kaggle.json ~/.kaggle/


! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download ananthu017/emotion-detection-fer
# ! kaggle datasets download emotion-detection-fer #For Localhost


In [ ]:
!pip install torchmetrics torchinfo GPUtil

In [ ]:
import requests
from zipfile import ZipFile
from pathlib import Path

import os
import glob

import random as random
from PIL import Image

import torch
from torch import nn

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import torchvision

from torchvision import datasets
from torchvision import transforms

import torchmetrics
import mlxtend

import matplotlib.pyplot as plt

from typing import Tuple
from typing import Dict
from typing import List
from timeit import default_timer as timer
from matplotlib import patches as mpatches

from tqdm.auto import tqdm

import gc

from numba import cuda

from GPUtil import showUtilization as gpu_usage


from torchinfo import summary

import shutil

import numpy as np
import pandas as pd

from os import listdir                  
from os.path import isfile, join

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
def extract_data(zipfile_path: Path, destination_path: Path) -> None:
    '''Extracts zipfile'''
    
    if destination_path.is_dir():
        print(f"{destination_path} exists.")
    else:
        print(f"{destination_path} doesn't exist, creating one...")
        destination_path.mkdir(parents=True, exist_ok=True)

    if not os.listdir(destination_path):
        with ZipFile(zipfile_path, 'r') as zip:
            print("Extracting files...")
            zip.extractall(destination_path)

            print("Extracting finished.")
    else:
        print("Data already extracted.")
    
data_path = Path("Kaggle/CV/Emotions_detection")
# zipfile_path = "/content/emotion-detection-fer.zip"
zipfile_path = "emotion-detection-fer.zip" #For Local use

images_path = data_path / "emotions_dataset"

extract_data(zipfile_path, images_path)

In [ ]:
%matplotlib inline

def print_random_image(images_path: list, seed=None) -> None:
    """Prints one random photo with details such as class, heigh, width"""
    if seed:
        random.seed(seed)

    random_image_path = random.choice(images_path)
    image_class = random_image_path.parent.stem
    image = Image.open(random_image_path)
    print(f"Random image class: {image_class}")
    print(f"Image height: {image.height}")
    print(f"Image width: {image.width}")
    plt.imshow(image.convert('P'))

    
image_path_list = list(images_path.glob("*/*/*"))
print_random_image(image_path_list)

In [ ]:
# preprocessing steps: 
# normalize values
# convert to grayscale
# create augmented data in an imagedatagenerator
# 
# transfer learning perhaps
# CNN with last layer being softmax# preprocessing steps: 
# normalize values
# convert to grayscale
# create augmented data in an imagedatagenerator
# 
# transfer learning perhaps
# CNN with last layer being softmax

# Preprocess Data

In [ ]:
train_transform = transforms.Compose(
    [transforms.Resize(size=(224, 224)),
     transforms.ToTensor()]
)

val_transform = transforms.Compose(
    [transforms.Resize(size=(224, 224)),
     transforms.ToTensor()]
)

In [ ]:
def plot_transformed_images(images_path: Path,
                            transform: transforms,
                            n: int=3,
                            seed=None) -> None:
    """Selects random images from a path, transforms them and plots original vs transform"""
    if seed:
        random.seed(seed)

    if n > 10:
        print("n shouldn't be higher than 10 due to the size of displayed plot, changing n to 10")
        n = 10

    random_image_paths = random.sample(images_path, k=n)

    for image_path in random_image_paths:
        with Image.open(image_path).convert('RGB') as f:
            fig, ax = plt.subplots(nrows=1, ncols=2)
            ax[0].imshow(f)
            ax[0].set_title(f"Original\nsize: {f.size}")
            ax[0].axis(False)

            transformed_image = transform(f).permute(1, 2, 0)
            ax[1].imshow(transformed_image)
            ax[1].set_title(f"Transformed\nshape: {transformed_image.shape}")
            ax[1].axis(False)


plot_transformed_images(image_path_list, train_transform)

In [ ]:
train_dir = images_path / "train"
val_dir = images_path / "test"

train_data = datasets.ImageFolder(
    root=train_dir,
    transform=train_transform
)

val_data = datasets.ImageFolder(
    root=val_dir,
    transform=val_transform
)

### Randomly shuffle the train and test images and simultaneously shuffle the respective labels in unison

In [ ]:
BATCH_SIZE = 8

train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

val_dataloader = DataLoader(
    dataset=val_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4
)

image_batch, label_batch = next(iter(train_dataloader))
image_batch.shape, label_batch.shape

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
output_shape = len(train_data.classes)

model = torchvision.models.efficientnet_b7().to(device)


In [ ]:
summary(
    model=model,
    input_size=(BATCH_SIZE, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)


In [ ]:

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=2560, 
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True)).to(device)




In [ ]:
def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device) -> Tuple[float, float]:

    model.train()
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        y_pred_class = torch.argmax(y_pred, dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc


def val_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: torch.device) -> Tuple[float, float, torch.Tensor]:

    model.eval()
    val_loss, val_acc = 0, 0
    y_preds = []

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            val_pred_logits = model(X)
            loss = loss_fn(val_pred_logits, y)
            val_loss += loss.item()

            val_pred_labels = torch.argmax(val_pred_logits, dim=1)
            val_acc += ((val_pred_labels == y).sum().item() / len(val_pred_labels))
            y_preds.append(val_pred_labels.cpu())

    val_loss = val_loss / len(dataloader)
    val_acc = val_acc / len(dataloader)

    y_pred_tensor = torch.cat(y_preds)

    return val_loss, val_acc, y_pred_tensor


def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    val_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs: int,
    device: torch.device) -> Tuple[Dict, torch.Tensor]:

    results = {"train_loss": [],
             "train_acc": [],
             "val_loss": [],
             "val_acc": []}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device
        )

        val_loss, val_acc, y_preds = val_step(
            model=model,
            dataloader=val_dataloader,
            loss_fn=loss_fn,
            device=device
        )

        print(f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Train acc: {train_acc:.3f}, Val loss: {val_loss:.3f}, Val acc: {val_acc:.3f}")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["val_loss"].append(val_loss)
        results["val_acc"].append(val_acc)

    return results, y_preds

In [ ]:
torch.cuda.manual_seed(42)
torch.manual_seed(42)

EPOCHS = 10

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(
    params=model.parameters(),
    lr=0.01
)

start_time = timer()

model_results, preds = train(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=EPOCHS,
    device=device
)

end_time = timer()
print(f"Total learning time: {(end_time - start_time):.3f}")

In [ ]:
def plot_curves(results: Dict[str, List[float]]) -> None:
    """Plots loss and accuracy from a results dictionary."""

    train_loss = results["train_loss"]
    val_loss = results["val_loss"]

    train_accuracy = results["train_acc"]
    val_accuracy = results["val_acc"]

    epochs = range(len(results["train_loss"]))

    plt.figure(figsize=(15, 7))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, label="train_loss")
    plt.plot(epochs, val_loss, label="val_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracy, label="train_accuracy")
    plt.plot(epochs, val_accuracy, label="val_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()


plot_curves(model_results)

In [ ]:
def make_predictions(model: torch.nn.Module,
                     data: list,
                     device: torch.device) -> torch.Tensor:

    pred_probs = []
    model.eval()

    with torch.inference_mode():
        for sample in data:
            sample = torch.unsqueeze(sample, dim=0).to(device)
            pred_logit = model(sample)
            pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)
            pred_probs.append(pred_prob.cpu())

    return torch.stack(pred_probs)


def show_predictions(model: torch.nn.Module,
                     device: torch.device,
                     val_data: datasets) -> None:

    val_samples = []
    val_labels = []

    for sample, label in random.sample(list(val_data), k=9):
        val_samples.append(sample)
        val_labels.append(label)

    pred_probs = make_predictions(
        model=model,
        data=val_samples,
        device=device
    )

    pred_classes = pred_probs.argmax(dim=1)
    plt.figure(figsize=(9, 9))
    nrows = 3
    ncols = 3

    for i, sample in enumerate(val_samples):
        plt.subplot(nrows, ncols, i+1)
        image = sample.squeeze().permute(1, 2, 0)
        plt.imshow(image)
        pred_label = val_data.classes[pred_classes[i]]
        truth_label = val_data.classes[val_labels[i]]
        title_text = f"Pred: {pred_label} | Truth: {truth_label}"

        if pred_label == truth_label:
            plt.title(title_text, fontsize=10, c="g")
        else:
            plt.title(title_text, fontsize=10, c="r")
        plt.axis(False)

In [ ]:

show_predictions(
    model=model,
    val_data=val_data,
    device=device)

In [ ]:
# Import/install Gradio 
try:
    import gradio as gr
except: 
    !pip -q install gradio
    import gradio as gr
    
print(f"Gradio version: {gr.__version__}")

In [ ]:

def predict(inp):
  inp = transforms.ToTensor()(inp).unsqueeze(0)
  with torch.no_grad():
    prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)
    confidences = {val_data.classes[i]: float(prediction[i]) for i in range(len(val_data.classes))}    
  return confidences

In [ ]:
model.to("cpu") 

# Check the device
next(iter(model.parameters())).device

In [ ]:
val_path_list = list(val_dir.glob("*/*"))
example_list = [[str(filepath)] for filepath in random.sample(val_path_list, k=3)]
example_list

#Save Model

In [ ]:
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.
    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.
    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

In [ ]:
save_model(model=model,
                 target_dir="models",
                 model_name="efficientnet_b0.pth")

# Deployment Test

In [ ]:
def create_effnetb0_model(num_classes:int=7, 
                          seed:int=42, reid=False):
    """Creates an EfficientNetB2 feature extractor model and transforms.

    Args:
        num_classes (int, optional): number of classes in the classifier head. 
            Defaults to 3.
        seed (int, optional): random seed value. Defaults to 42.

    Returns:
        model (torch.nn.Module): EffNetB0 feature extractor model. 
        transforms (torchvision.transforms): EffNetB0 image transforms.
    """
    # Create EffNetB0 pretrained weights, transforms and model
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b0(weights=weights)

    # Freeze all layers in base model
    for param in model.parameters():
        param.requires_grad = False

    # Change classifier head with random seed for reproducibility
    torch.manual_seed(seed)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1280, out_features=num_classes),
    )
    
    return model, transforms

In [ ]:
import torchvision.transforms 
class_names = ["Happy", "Sad", "Disgusted","Suprised","Fearful","Angry","Neutral"]

### 2. Model and transforms preparation ###

# Create EffNetB2 model
effnetb0, effnetb0_transforms = create_effnetb0_model(
    num_classes=7, # len(class_names) would also work
)

# Load saved weights
effnetb0.load_state_dict(
    torch.load(
        f="models/efficientnet_b0.pth",
        map_location=torch.device("cpu"),  # load to CPU
    )
)

### 3. Predict function ###

# Create predict function

def predict(inp):
  inp = transforms.ToTensor()(inp).unsqueeze(0)
  with torch.no_grad():
    prediction = torch.nn.functional.softmax(effnetb0(inp)[0], dim=0)
    confidences = {class_names[i]: float(prediction[i]) for i in range(len(class_names))}    
  return confidences


In [ ]:
### 4. Gradio app ###

# Create title, description and article strings
title = "Emotion Detection App 😀😐😰😞🤢😲😡"
description = "An EfficientNetB0 computer vision model to classify images of emotions: Happy, Neutral, Sad, fearful, Angry, Suprised, Disgusted."
article = "Reference: [09. PyTorch Model Deployment](https://www.learnpytorch.io/09_pytorch_model_deployment/)."

import gradio as gr

gr.Interface(fn=predict, 
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=7),
             examples=example_list,
                             title=title,
                    description=description,
                    article=article).launch()


# Deploying Model to Production

In [ ]:
import shutil
from pathlib import Path

# Create FoodVision mini demo path
foodvision_mini_demo_path = Path("demos/emotiondetection_app/")

# Remove files that might already exist there and create new directory
if foodvision_mini_demo_path.exists():
    shutil.rmtree(foodvision_mini_demo_path)
    foodvision_mini_demo_path.mkdir(parents=True, # make the parent folders?
                                    exist_ok=True) # create it even if it already exists?
else:
    # If the file doesn't exist, create it anyway
    foodvision_mini_demo_path.mkdir(parents=True, 
                                    exist_ok=True)
    
# Check what's in the folder
!ls demos/emoitiondetection_app/

In [ ]:
import shutil
from pathlib import Path

# 1. Create an examples directory
foodvision_mini_examples_path = foodvision_mini_demo_path / "examples"
foodvision_mini_examples_path.mkdir(parents=True, exist_ok=True)

# 2. Collect three random test dataset image paths
foodvision_mini_examples = [Path('Kaggle/CV/Emotions_detection/emotions_dataset/test/fearful/im867.png'),
                            Path('Kaggle/CV/Emotions_detection/emotions_dataset/test/sad/im90.png'),
                            Path('Kaggle/CV/Emotions_detection/emotions_dataset/test/fearful/im175.png')]

# 3. Copy the three random images to the examples directory
for example in foodvision_mini_examples:
    destination = foodvision_mini_examples_path / example.name
    print(f"[INFO] Copying {example} to {destination}")
    shutil.copy2(src=example, dst=destination)

In [ ]:
import os

# Get example filepaths in a list of lists
example_list = [["examples/" + example] for example in os.listdir(foodvision_mini_examples_path)]
example_list

In [ ]:
import shutil

# Create a source path for our target model
effnetb0_foodvision_mini_model_path = "models/efficientnetb0.pth"

# Create a destination path for our target model 
effnetb0_foodvision_mini_model_destination = foodvision_mini_demo_path / effnetb0_foodvision_mini_model_path.split("/")[1]

# Try to move the file
try:
    print(f"[INFO] Attempting to move {effnetb0_foodvision_mini_model_path} to {effnetb0_foodvision_mini_model_destination}")
    
    # Move the model
    shutil.move(src=effnetb0_foodvision_mini_model_path, 
                dst=effnetb0_foodvision_mini_model_destination)
    
    print(f"[INFO] Model move complete.")

# If the model has already been moved, check if it exists
except:
    print(f"[INFO] No model found at {effnetb0_foodvision_mini_model_path}, perhaps its already been moved?")
    print(f"[INFO] Model exists at {effnetb0_foodvision_mini_model_destination}: {effnetb0_foodvision_mini_model_destination.exists()}")

In [ ]:
%%writefile demos/emotiondetection_app//model.py
import torch
import torchvision

from torch import nn


def create_effnetb0_model(num_classes:int=7, 
                          seed:int=42):
    """Creates an EfficientNetB2 feature extractor model and transforms.

    Args:
        num_classes (int, optional): number of classes in the classifier head. 
            Defaults to 3.
        seed (int, optional): random seed value. Defaults to 42.

    Returns:
        model (torch.nn.Module): EffNetB0 feature extractor model. 
        transforms (torchvision.transforms): EffNetB0 image transforms.
    """
    # Create EffNetB0 pretrained weights, transforms and model
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b0(weights=weights)

    # Freeze all layers in base model
    for param in model.parameters():
        param.requires_grad = False

    # Change classifier head with random seed for reproducibility
    torch.manual_seed(seed)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes),
    )
    
    return model, transforms

In [ ]:
%%writefile demos/emotiondetection_app/app.py
### 1. Imports and class names setup ### 
import gradio as gr
import os
import torch

from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple, Dict

# Setup class names
class_names = ["Happy", "Sad", "Disgusted","fearful","Angry","Neutral"]

### 2. Model and transforms preparation ###

# Create EffNetB2 model
effnetb0, effnetb0_transforms = create_effnetb0_model(
    num_classes=7, # len(class_names) would also work
)

# Load saved weights
effnetb0.load_state_dict(
    torch.load(
        f="models/efficientnet_b0.pth",
        map_location=torch.device("cpu"),  # load to CPU
    )
)

### 3. Predict function ###

# Create predict function

def predict(inp):
  inp = transforms.ToTensor()(inp).unsqueeze(0)
  with torch.no_grad():
    prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)
    confidences = {val_data.classes[i]: float(prediction[i]) for i in range(len(val_data.classes))}    
  return confidences


### 4. Gradio app ###

# Create title, description and article strings
title = "Emotion Detection App 😀😐😰😞🤢😲😡"
description = "An EfficientNetB0 computer vision model to classify images of emotions: Happy, Neutral, Sad, fearful, Angry, Suprised, Disgusted."
article = "Reference: [09. PyTorch Model Deployment](https://www.learnpytorch.io/09_pytorch_model_deployment/)."

import gradio as gr

gr.Interface(fn=predict, 
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=7),
             examples=example_list,
                             title=title,
                    description=description,
                    article=article).launch()

In [ ]:
%%writefile demos/emotiondetection_app/requirements.txt
torch==1.12.0
torchvision==0.1.9
gradio==3.1.4

In [ ]:
!ls demos/emotiondetection_app

In [ ]:
# Change into and then zip the foodvision_mini folder but exclude certain files
!cd demos/emotiondetection_app && zip -r ../emotiondetection_app.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

# Download the zipped FoodVision Mini app (if running in Google Colab)
try:
    from google.colab import files
    files.download("demos/emotion_detection.zip")
except:
    print("Not running in Google Colab, can't use google.colab.files.download(), please manually download.")

# References

* https://www.kaggle.com/datasets/ananthu017/emotion-detection-fer

* https://debuggercafe.com/pytorch-pretrained-efficientnet-model-image-classification/

* https://www.learnpytorch.io/05_pytorch_going_modular/

* https://github.com/mrdbourke/pytorch-deep-learning/blob/main/03_pytorch_computer_vision.ipynb

* https://www.kaggle.com/code/saworz/animals-classification-pretrained-vgg16-val94-5

* https://pytorch.org/vision/main/models/generated/torchvision.models.efficientnet_b0.html

* To run on your local machine/server:: https://research.google.com/colaboratory/local-runtimes.html
